# HFT - Intraday volatility estimation from high frequency data


**Lucas RODRIGUEZ**

*Academic work - October/November/December 2022*

In [1]:
import pandas as pd

In [7]:
data_folder = "data/"
source_filename = data_folder + "IVE_tickbidask.txt"

columns_name = ["Date", "Time", "Price", "Bid", "Ask", "Size"]
df = pd.read_csv(filepath_or_buffer=source_filename,
                names=columns_name,
                decimal=".",
                parse_dates={'dt' : ['Date', 'Time']}, 
                infer_datetime_format=True, 
                low_memory=False, 
                index_col='dt')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10383278 entries, 2009-09-28 09:30:00 to 2022-10-28 16:00:00
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Price   float64
 1   Bid     float64
 2   Ask     float64
 3   Size    int64  
dtypes: float64(3), int64(1)
memory usage: 396.1 MB


In [9]:
df.head(10)

,Price,Bid,Ask,Size
dt,,,,
2009-09-28 09:30:00,50.79,50.70,50.79,100
2009-09-28 09:30:00,50.71,50.70,50.79,638
2009-09-28 09:31:32,50.75,50.75,50.76,100
2009-09-28 09:31:32,50.75,50.75,50.76,100
2009-09-28 09:31:33,50.75,50.75,50.76,100
2009-09-28 09:31:33,50.75,50.75,50.76,100
2009-09-28 09:31:33,50.75,50.75,50.76,100
2009-09-28 09:31:33,50.75,50.75,50.76,100
2009-09-28 09:31:33,50.75,50.75,50.76,100


In [12]:
df.resample("30S").mean()

,Price,Bid,Ask,Size
dt,,,,
2009-09-28 09:30:00,50.750000,50.700000,50.790000,369.000000
2009-09-28 09:30:30,NaN,NaN,NaN,NaN
2009-09-28 09:31:00,NaN,NaN,NaN,NaN
2009-09-28 09:31:30,50.750000,50.742500,50.758333,133.333333
2009-09-28 09:32:00,50.780000,50.760000,50.780000,300.000000
...,...,...,...,...
2022-10-28 15:58:00,143.812692,143.803791,143.813352,155.296703
2022-10-28 15:58:30,143.812031,143.803594,143.817969,152.984375
2022-10-28 15:59:00,143.893206,143.885115,143.896870,236.488550
